In [1]:
"""Loading data with LUSID Python Tools

Demonstrates how to load portfolios, instruments, holdings, and transactions.

Attributes
----------
instruments
transactions
portfolios
cocoon
"""

'Loading data with LUSID Python Tools\n\nDemonstrates how to load portfolios, instruments, holdings, and transactions.\n\nAttributes\n----------\ninstruments\ntransactions\nportfolios\ncocoon\n'

# Import your Libraries

In [2]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken
import fbnsdkutilities.utilities as utils

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import requests
import os
import lusid
import lusidtools.cocoon.cocoon as cocoon
import uuid

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = utils.ApiClientFactory(
    lusid,
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

scope = 'my_test_scope_12'

print ('LUSID Environment Initialised')
print ('LUSID API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.6.11244.0


# 1) Create Portfolios

## a) Import portfolio data

In [3]:
positions_file = pd.read_csv('./data/Sample_Positions.csv')
positions_file.head()

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD


## b) Apply data transformations specific to your data

In [4]:
portfolios_data = pd.DataFrame(positions_file['FundCode'])
portfolios_data['display_name'] = portfolios_data['FundCode'].apply(lambda x: "Fund {}".format(x))
portfolios_data['created'] = "2010-10-09T08:00:00Z"
portfolios_data['base_currency'] = 'GBP'
portfolios_data['description'] = portfolios_data['FundCode'].apply(lambda x: "The fund with the fund code {}".format(x))
portfolios_data.head()

,FundCode,display_name,created,base_currency,description
0,Portfolio-Z,Fund Portfolio-Z,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-Z
1,Portfolio-Z,Fund Portfolio-Z,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-Z
2,Portfolio-Z,Fund Portfolio-Z,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-Z
3,Portfolio-X,Fund Portfolio-X,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-X
4,Portfolio-X,Fund Portfolio-X,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-X


## c) Set your portfolio field mapping

In [5]:
portfolio_mapping_required = {
  'code': 'FundCode',
  'display_name': 'display_name',
  'base_currency': 'base_currency'
}

portfolio_mapping_optional = {
  'description': 'description',
  'accounting_method': None,
  'created': 'created'
}

## d) Create your portfolios

In [6]:
responses = cocoon.load_from_data_frame(
    api_factory=api_factory, 
    scope=scope, 
    data_frame=portfolios_data, 
    mapping_required=portfolio_mapping_required, 
    mapping_optional=portfolio_mapping_optional,
    file_type='portfolios')

for response in responses['portfolios']['success']:
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: my_test_scope_12
Code: Portfolio-Z
Portfolio Effective From: 2010-10-09 08:00:00+00:00
Portfolio Created On: 2023-05-04 11:50:04.107845+00:00

Portfolio Created
Scope: my_test_scope_12
Code: Portfolio-X
Portfolio Effective From: 2010-10-09 08:00:00+00:00
Portfolio Created On: 2023-05-04 11:50:04.107845+00:00



# 2) Load your instrument universe

## a) Import instrument data

In [7]:
positions_file

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD


## b) Apply data transformations specific to your data

In [8]:
instruments_data = pd.DataFrame(
    positions_file.loc[:, [
        'Local Currency Code', 
        'Security Description',
        'ISIN Security Identifier',
        'SEDOL Security Identifier']]
    )

instruments_data

,Local Currency Code,Security Description,ISIN Security Identifier,SEDOL Security Identifier
0,GBP,Lloyds Banking Group PLC,GB0008706128,870612
1,USD,Apple Inc,US0378331005,0865985
2,USD,Amazon,US0231351067,906866
3,GBX,Sainsbury,XS1139087933,B019KW7
4,USD,Tesla Inc,US88160R1014,BSJC712


## c) Set your instruments field mapping

In [9]:
instrument_mapping_required = {
  'name': 'Security Description'
}

instrument_identifier_mapping = {
    'ClientInternal': 'SEDOL Security Identifier',
    'Sedol': 'SEDOL Security Identifier',
    'Isin': 'ISIN Security Identifier'
}

instrument_mapping_optional = {
  'definition': None
}

## d) Upsert your instruments

In [10]:
response = cocoon.load_from_data_frame(
    api_factory=api_factory, 
    data_frame=instruments_data, 
    identifier_mapping=instrument_identifier_mapping, 
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type='instruments',
    scope='InstrumentProperties001')

prettyprint.instrument_response(response['instruments']['success'][0])

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,ClientInternal: BSJC712,BSJC712,LUID_RN7XCUIN
1,ClientInternal: 0865985,0865985,LUID_T0D62072
2,ClientInternal: B019KW7,B019KW7,LUID_N5B98K9D
3,ClientInternal: 906866,906866,LUID_44XTCRUQ
4,ClientInternal: 870612,870612,LUID_XZRQZ584


# 3) Set your holdings

## a) Import holdings data

In [11]:
holdings_file = positions_file.copy(deep=True)
holdings_file

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD


## b) Apply data transformations specific to your data

In [12]:
holdings_file['is_cash_with_currency'] = np.NaN

# Convert amounts in Pence to be in Pounds
for column in ["Local Price", "Local Market Value",  "Base Market Value"]:
    holdings_file[column] = holdings_file.apply(
        lambda x: getattr(x,column)/100 if getattr(x, "Local Currency Code") == "GBX" else getattr(x,column), axis=1)
    
holdings_file['Local Currency Code'] = holdings_file['Local Currency Code'].apply(
    lambda x: "GBP" if x == "GBX" else x)
holdings_file.head()

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency,is_cash_with_currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900.00,141900.00,Lloyds Banking Group PLC,GB0008706128,870612,GBP,NaN
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000.00,2084000.00,Apple Inc,US0378331005,0865985,USD,NaN
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721.00,52916721.00,Amazon,US0231351067,906866,USD,NaN
3,Portfolio-X,05/02/2019,BAML,GBP,942354,2.00,1889419.77,1889419.77,Sainsbury,XS1139087933,B019KW7,GBP,NaN
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434.00,22038434.00,Tesla Inc,US88160R1014,BSJC712,USD,NaN


## c) Set your holdings field mapping

In [13]:
holding_mapping_required = {
  'code': 'FundCode',
  'effective_at': 'Effective Date',
  'tax_lots.units': 'Quantity'
}

holding_identifier_mapping = {
  'Isin': 'ISIN Security Identifier',
  'Sedol': 'SEDOL Security Identifier',
  'ClientInternal': 'SEDOL Security Identifier',
  'Currency': 'is_cash_with_currency'
}

holding_mapping_optional= {
  'tax_lots.cost.amount': None,
  'tax_lots.cost.currency': "Local Currency Code",
  'tax_lots.portfolio_cost': None,
  'tax_lots.price': None,
  'tax_lots.purchase_date': None,
  'tax_lots.settlement_date': None
}

## d) Add your holdings

In [14]:
responses = cocoon.load_from_data_frame(
    api_factory=api_factory, 
    scope=scope, 
    data_frame=holdings_file,
    mapping_required=holding_mapping_required, 
    mapping_optional=holding_mapping_optional,
    identifier_mapping=holding_identifier_mapping,
    file_type='holding')

for response in responses['holdings']['success']:
    print ('\n')
    prettyprint.adjust_holdings_response(response, scope, response.href.split("/")[7])



Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_12
Code: Portfolio-Z
Adjusted Holdings Effective From: 2010-10-09 08:00:00+00:00
Adjusted Holdings Created On: 2023-05-04 11:50:14.872015+00:00



Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_12
Code: Portfolio-Z
Adjusted Holdings Effective From: 2010-10-09 08:00:00+00:00
Adjusted Holdings Created On: 2023-05-04 11:50:19.537631+00:00



Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_12
Code: Portfolio-Z
Adjusted Holdings Effective From: 2010-10-09 08:00:00+00:00
Adjusted Holdings Created On: 2023-05-04 11:50:22.770034+00:00



Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_12
Code: Portfolio-X
Adjusted Holdings Effective From: 2010-10-09 08:00:00+00:00
Adjusted Holdings Created On: 2023-05-04 11:50:23.570662+00:00



Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_12
Code: Portfolio-X
Adjusted Holdings Effective From: 2010-10-09 08:00:00+00:00

In [15]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code="Portfolio-X",
    property_keys=["Instrument/default/Name"])

holdings = prettyprint.get_holdings_df(response)
holdings

,local_vars_configuration,_instrument_scope,_instrument_uid,_holding_type,_units,_settled_units,_transaction,_currency,_holding_type_name,discriminator,cost.amount,cost.currency,cost_portfolio_ccy.amount,Instrument/default/Name,Holding/default/SourcePortfolioId,Holding/default/SourcePortfolioScope
0,<lusid.configuration.Configuration object at 0...,default,LUID_N5B98K9D,P,942354.00,942354.00,None,GBP,Position,None,0.00,GBP,0.00,Sainsbury,Portfolio-X,my_test_scope_12


In [16]:
holdings.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   local_vars_configuration              1 non-null      object 
 1   _instrument_scope                     1 non-null      object 
 2   _instrument_uid                       1 non-null      object 
 3   _holding_type                         1 non-null      object 
 4   _units                                1 non-null      float64
 5   _settled_units                        1 non-null      float64
 6   _transaction                          0 non-null      object 
 7   _currency                             1 non-null      object 
 8   _holding_type_name                    1 non-null      object 
 9   discriminator                         0 non-null      object 
 10  cost.amount                           1 non-null      float64
 11  cost.currency          

# 4) Add your transactions

## a) Import transactions data

In [17]:
transactions_data = positions_file.copy(deep=True)
transactions_data

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD


## b) Apply data transformations specific to your data

In [18]:
transactions_data['transaction_type'] = 'FundsIn'
transactions_data['exchange_rate'] = 1
transactions_data['is_cash_with_currency'] = np.NaN

# Convert amounts in Pence to be in Pounds
for column in ["Local Price", "Local Market Value",  "Base Market Value"]:
    transactions_data[column] = transactions_data.apply(
        lambda x: getattr(x,column)/100 if getattr(x, "Local Currency Code") == "GBX" else getattr(x,column), axis=1)
transactions_data['Local Currency Code'] = transactions_data['Local Currency Code'].apply(
    lambda x: "GBP" if x == "GBX" else x)

transactions_data['transaction_id'] = [str(uuid.uuid4()) for _ in range(len(transactions_data.index))]
transactions_data['Effective Date'] = pd.to_datetime(
    transactions_data['Effective Date'], format="%d/%m/%Y").apply(lambda x: pytz.utc.localize(x))
transactions_data.head()

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency,transaction_type,exchange_rate,is_cash_with_currency,transaction_id
0,Portfolio-Z,2019-07-02 00:00:00+00:00,UBS,GBP,2500,56.76,141900.00,141900.00,Lloyds Banking Group PLC,GB0008706128,870612,GBP,FundsIn,1,NaN,e46cf434-cd2b-425b-ab15-7ed875c56384
1,Portfolio-Z,2019-01-01 00:00:00+00:00,CITI,USD,10000,208.40,2084000.00,2084000.00,Apple Inc,US0378331005,0865985,USD,FundsIn,1,NaN,cdd57ca9-ad9e-4fe2-9b4f-d5a7ad1a3cb5
2,Portfolio-Z,2018-12-01 00:00:00+00:00,UBS,USD,26598,1989.50,52916721.00,52916721.00,Amazon,US0231351067,906866,USD,FundsIn,1,NaN,7c16b94f-ba49-4cce-9d0f-ce6dbbbf07c5
3,Portfolio-X,2019-02-05 00:00:00+00:00,BAML,GBP,942354,2.00,1889419.77,1889419.77,Sainsbury,XS1139087933,B019KW7,GBP,FundsIn,1,NaN,37623611-c5ed-4d19-8b58-f5bb022fa93d
4,Portfolio-X,2018-11-24 00:00:00+00:00,MS,USD,95421,230.96,22038434.00,22038434.00,Tesla Inc,US88160R1014,BSJC712,USD,FundsIn,1,NaN,b3635077-f089-40e6-949e-d399965c1ef7


## c) Set your transactions field mapping

In [19]:
transaction_field_mapping_required = {
    "code": "FundCode",
    "transaction_id": "transaction_id",
    "type": 'transaction_type',
    "transaction_date": 'Effective Date',
    "settlement_date": 'Effective Date',
    "units": "Quantity",
    "transaction_price.price": "Local Price",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "Local Market Value",
    "total_consideration.currency": "Local Currency Code",
    "transaction_currency": "Local Currency Code"
    }

transaction_field_mapping_optional = {
    "exchange_rate": "exchange_rate",
    "source": "$Client"
}

transaction_identifier_mapping = {
  'Isin': 'ISIN Security Identifier',
  'Sedol': 'SEDOL Security Identifier',
  'ClientInternal': 'SEDOL Security Identifier',
  "Currency": 'is_cash_with_currency'
    
}

## d) Add your transactions

In [20]:
responses = cocoon.load_from_data_frame(
    api_factory=api_factory, 
    scope=scope, 
    data_frame=transactions_data,
    mapping_required=transaction_field_mapping_required, 
    mapping_optional=transaction_field_mapping_optional,
    identifier_mapping=transaction_identifier_mapping,
    file_type='transaction')

for response in responses['transactions']['success']:
    print ('\n')
    prettyprint.transactions_response(response, scope, response.href.split("/")[7])



Transactions Successfully Upserted into Portfolio
Scope: my_test_scope_12
Code: Portfolio-Z
Transactions Effective From: 2019-07-02 00:00:00+00:00
Transactions Created On: 2023-05-04 11:50:33.790568+00:00



Transactions Successfully Upserted into Portfolio
Scope: my_test_scope_12
Code: Portfolio-X
Transactions Effective From: 2019-02-05 00:00:00+00:00
Transactions Created On: 2023-05-04 11:50:31.054322+00:00



In [21]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code="Portfolio-X",
    property_keys=["Instrument/default/Name"])

holdings = prettyprint.get_holdings_df(response)
holdings

,local_vars_configuration,_instrument_scope,_instrument_uid,_holding_type,_units,_settled_units,_transaction,_currency,_holding_type_name,discriminator,cost.amount,cost.currency,cost_portfolio_ccy.amount,Instrument/default/Name,Holding/default/SourcePortfolioId,Holding/default/SourcePortfolioScope
0,<lusid.configuration.Configuration object at 0...,default,LUID_N5B98K9D,P,942354.00,942354.00,None,GBP,Position,None,0.00,GBP,0.00,Sainsbury,Portfolio-X,my_test_scope_12
